*This jupyter notebook is part of Arizona State University's course CAS 523 (Methods for Complex Systems Science: Statistics and Dimensionality Reduction) and was written by Bryan Daniels.  It was last updated August 24, 2022.*

*This assignment uses data from the [Stanford Network Analysis Project](http://snap.stanford.edu/) (SNAP).  More information about each data set is included in links below.*

# Interpreting network statistics

We quite often use the concept of a network to represent the structure of complex systems.  Once we have such a representation, statistics that capture important information about the network structure can be useful in understanding the system's behavior.

In this exercise, we will practice loading existing network data from two social systems, interpreting statistics measured from these networks, and doing some basic dimensionality reduction for visualization.

## Load python packages and bibliographic data

First we load some standard packages:

In [ ]:
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 18}) # increases font size on plots
from pathlib import Path # to handle file paths across all operating systems

For network analysis, we will use the popular python package `networkx` (see [networkx.org](https://networkx.org/) for more information about this package):

In [ ]:
import networkx as nx
import networkx.algorithms.community as nx_comm # community detection functionality
from helpers.networkx_patch import * # temporary patch for networkx 2.7.1 under scipy 1.7.3

We will first study a network representing collaborations between physicists studying general relativity and quantum cosmology.  Two physicists are connected by an edge in the network if they appeared as co-authors on an academic paper between January 1993 and April 2003.  Details about this dataset can be found [here](http://snap.stanford.edu/data/ca-GrQc.html).

The function `read_edgelist` reads in a text file in which each line in the file represents an edge in a network.  The result is a `networkx` `Graph` object.

In [ ]:
G = nx.read_edgelist(Path('data/SNAP/ca-GrQc.txt'))

In [ ]:
type(G)

We can get some basic information about the network to check that the file loaded correctly:

In [ ]:
print("The network contains {} nodes and {} edges.".format(G.number_of_nodes(),G.number_of_edges()))

❓ **For the scientists and timeframe covered by this dataset, what is the average number of other scientists that each scientist has co-authored with?** *Hint: Each edge (representing a co-authorship) is attached to two nodes (representing scientists).*

✳️ **Answer:** 

## Network visualization

Let's try to visualize this network.  Visualization of large networks is generally a hard problem and could easily be the subject of its own course.  We will use the dimensionality reduction technique of clustering (also known as "community detection" in the network science literature) to try to make some progress.

By default, `nx.draw` will use a so-called "spring" layout, where nodes are initially placed at random, and edges are treated as springs that have some preferred length.  Nodes are then moved around by the algorithm so that edges are closer to their preferred length.

Try running `nx.draw` for our network below.  For a network of this size, the algorithm will take a few minutes to run.

In [ ]:
plt.figure(figsize=(20,20))
nx.draw(G)

The results are somewhat random due to the initial random placement, but it is likely that you got one large blob of nodes in the center surrounded by smaller blobs.  This is showing the fact that there is one large "connected component" of scientists and many other smaller groups, but we don't get much else out of this plot.

(By the way: you may also notice some circular "self-loops" in the network.  This is coming from the fact that, for some reason, a few authors are listed as having co-authored with themselves.  This is a good example of an anomaly in the data that we should check into to make sure we understand what's going on.  For now, we will just ignore these self-loops as there aren't that many, but in a real use case it would make sense to ask the original collectors of the data about this.  You can also remove the self-loops using `G.remove_edges_from(nx.selfloop_edges(G))` if you want to have cleaner-looking visualizations.)

Now let's use ideas of dimensionality reduction to find more structure.  Specifically, we will use a community detection algorithm to split the scientists into related groups.  The function `louvain_communities` is one such algorithm.

In [ ]:
communities = nx_comm.louvain_communities(G)

Let's see what this function produces (and remember that you can use shift-Tab in Jupyter to look at the function's documentation):

In [ ]:
len(communities)

In [ ]:
communities[0]

In [ ]:
community_sizes = [ len(c) for c in communities ]
community_sizes[:10]

So we have about 400 communities of varying sizes, with each community specified by a list of nodes.

Let's redraw the network with these communities plotted in different colors.  We'll use the function `nx.draw_networkx`, which gives us more control over the plot than the simple `nx.draw` (I also reduced the size of the nodes by setting `node_size` and the width of the edges using `width`):

In [ ]:
# makes a dictionary mapping each node to a color corresponding to its community
colors_dict = dict(np.concatenate([ list(zip(comm,np.repeat('C{}'.format(i),len(comm)))) \
                                    for i,comm in enumerate(communities) ]))

# makes network plot using a different color for each community
plt.figure(figsize=(20,20))
nx.draw_networkx(G,
                 node_color=[ colors_dict[node] for node in G.nodes ],
                 with_labels=False,
                 node_size=10,
                 width=0.1)
plt.axis('off')
plt.axis('equal');

We're getting closer to something more interpretable, but the blob in the center is still just a blob.  Let's plot just a few communities at a time to make things easier to see.

The following function creates a subgraph that includes only nodes that are in communities with indices given in `community_indices`.  For instance, `community_subgraph(G,communities,[2,3])` will produce a subgraph of `G` that only includes nodes in communities 2 and 3.

In [ ]:
def community_subgraph(G,communities,community_indices):
    communities_subset = [communities[i] for i in community_indices]
    return G.subgraph(set.union(*communities_subset))

❓ **Use `community_subgraph` and `draw_networkx` as above to make a network diagram that includes just two (or a few) large communities.  Call your subgraph `subG` and keep it for later use.** *Hint: Given the random initial conditions in the network layout routine, you may want to run the network plotting a few times to find a visualization that is easiest to interpret.  Feel free to play with the arguments of `draw_networkx` to get a picture that you are happy with.*

In [ ]:
# ✳️ **Answer:**

❓ **Interpret your results in terms of collaboration structures.  How does the community detection algorithm help with interpretation?** *Hint: What does `louvain_communities` look for in the network structure?  It may help to look at the documentation and to recall the typical goal of community detection algorithms.*

✳️ **Answer:** 

## Compute large-scale statistics

The following code computes the average shortest path length within the subgraph `subG` and the average clustering coefficient in the entire network:

In [ ]:
nx.average_shortest_path_length(subG)

In [ ]:
nx.average_clustering(G)

❓ **Interpret these statistics in terms of scientific collaborations.**

✳️ **Answer:** 

## Plot and roughly fit the degree distribution

Finally, we will look at the distribution of degrees.  The following code plots the empirically observed degree distribution on a log-log plot:

In [ ]:
# create a list of the degree of each node
# (this code filters out degrees of zero)
degree_list = [d for n, d in G.degree() if d > 0]
# count the number of nodes that have each degree
degrees,counts = np.unique(degree_list,return_counts=True)

# make a plot
plt.scatter(degrees,counts)
plt.xlabel('Degree')
plt.ylabel('Frequency')
plt.xscale('log')
plt.yscale('log')

The shape of this distribution can be very roughly fit by a Pareto (power-law) distribution.  The following code computes the best fit Pareto distribution and plots it alongside the data:

In [ ]:
paramsPowerLaw = stats.pareto.fit(degree_list,floc=0,fscale=1)
print(paramsPowerLaw)

In [ ]:
xs = np.linspace(1,100,100)
plt.plot(xs,[stats.pareto.pdf(x,*paramsPowerLaw) for x in xs],color='C1')
plt.scatter(degrees,counts/np.sum(counts),zorder=2,label='Power law fit')
plt.xlabel('Degree')
plt.ylabel('Probability density')
plt.xscale('log')
plt.yscale('log')

(Note: There is some controversy in the complex systems research literature about whether power law (aka "scale free") distributions are truly ubiquitous or merely overused.  Some have complained that any distribution that looks close to straight on a log-log plot is automatically assumed to be a power-law, even when it is not a particularly convincing fit.  We won't explore this in much detail here, but a good entry point into this literature can be found [here](https://doi.org/10.1137/070710111).)

❓ **How might this degree distribution analysis help us in thinking about scientific collaborations?  Could it help in building a model of scientific collaborations or innovations?**

✳️ **Answer:** 

⚛️ **Bonus question (for nothing but bragging rights): Do the same degree distribution analysis for the social network dataset from Google Plus in 2012, available [here](http://snap.stanford.edu/data/ego-Gplus.html), which includes the friendship network of over 100,000 people.** *Hint: The data set takes a few minutes to load, but after that the relevant computations run relatively quickly on my laptop.*

✴️ **Answer:**